## OLD vresion, needs updating

In [1]:
import pandas as pd

processed_regs_file = "./regs/gdpr.csv"
df = pd.read_csv(processed_regs_file, sep="|", encoding="utf-8", na_filter=False)

In [15]:
from openai import OpenAI
openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"),)

import importlib
import src.valid_index
importlib.reload(src.valid_index)
from src.valid_index import get_gdpr_index, ValidIndex

reference_checker = get_gdpr_index()

import src.embeddings
importlib.reload(src.embeddings)
from src.embeddings import num_tokens_from_string, get_ada_embedding

import src.file_tools
importlib.reload(src.file_tools)
from src.file_tools import get_regulation

import src.summarise_and_question
importlib.reload(src.summarise_and_question)
from src.summarise_and_question import get_summary_and_questions_for



ModuleNotFoundError: No module named 'src'

In [3]:
first_time = False
#sectioned_df = pd.DataFrame([],columns = ["article", "text", "token_count"])
save_sectioned_df_to_file = "./tmp/manual.csv"


if first_time:
    print("Loading the initial split of the tree. You will need to make changes to this as you see the data")
    sections_as_list = []
    counter = 1    
    unique_article_numbers = df["article_number"].unique()
    for article in unique_article_numbers:
        if article != counter:
            print(f"Article {article} should be number {counter}")
            break
        regs = get_regulation(df = df, index = str(article), reference_checker = index_checker, include_section_and_chapter=False)
        num_tokens = num_tokens_from_string(regs)
        sections_as_list.append([article, regs, num_tokens])
        counter += 1

    sectioned_df = pd.DataFrame(sections_as_list, columns = ["article", "text", "token_count"])
    sectioned_df.to_csv(save_sectioned_df_to_file, encoding="utf-8", sep="|", index = False)
else:
    print("Loading the current split of the tree so you can continue generating summaries and questions")
    sectioned_df = pd.read_csv(save_sectioned_df_to_file, encoding="utf-8", sep="|")

print(f'Total number of sections: {len(sectioned_df)}')


Loading the current split of the tree so you can continue generating summaries and questions
Total number of sections: 99


In [5]:
article_summary_with_embeddings = "./tmp/summary_gdpr_with_embedding.parquet"
article_questions_with_embeddings = "./tmp/questions_gdpr_with_embedding.parquet"
article_headings_index_file = "./tmp/headings.csv"

In [6]:
df_summary = None
if os.path.exists(article_summary_with_embeddings):
    df_summary = pd.read_parquet(article_summary_with_embeddings, engine='pyarrow')
    print(f"Summary data contains {len(df_summary)} lines of text")
    missing = len(df_summary[df_summary["text"] == ""])
    if missing > 0:
        print(f" -- of which there are {missing} lines that do not contain index text (e.g. sections with only definitions or indexes)")
else:
    print("Creating a new summary DataFrame")
    df_summary = pd.DataFrame([], columns = ["text", "article"])


df_questions = None
if os.path.exists(article_questions_with_embeddings):
    df_questions = pd.read_parquet(article_questions_with_embeddings, engine='pyarrow')
    print(f"Questions data contains {len(df_questions)} lines of text")    
    missing = len(df_questions[df_questions["text"] == ""])
    if missing > 0:
        print(f" -- of which there are {missing} lines that do not contain index text (e.g. sections with only definitions or indexes)")
else:
    print("Creating a new questions DataFrame")
    df_questions = pd.DataFrame([], columns = ["text", "article"])

index = None
if len(df_summary) != len(df_questions):
    print("The summary and the questions DataFrames do not have the same length")
else:
    index = len(df_summary)
    p = (index / len(sectioned_df)) * 100
    print(f'There are a total number of {len(sectioned_df)} sections to index')
    print(f"You have created {p:.2f} percent of your text index")

Summary data contains 64 lines of text
Questions data contains 64 lines of text
There are a total number of 99 sections to index
You have created 64.65 percent of your text index


In [7]:
index = index + 1

In [92]:
#model = "gpt-3.5-turbo"
#model="gpt-4"
model = "gpt-4-0125-preview"

reg_text = sectioned_df.loc[index]['text']
print("##############")
print(reg_text)
print("##############")

model_summary, model_questions = get_summary_and_questions_for(openai_client = openai_client, text = reg_text, model = model)

#format output
article = sectioned_df.loc[index]['article']
print(f'df_summary.loc[index, "article"] = "{article}"')
print(f'df_summary.loc[index, "text"] = "{model_summary}"')
print()
print(f'df_questions.loc[index, "article"] = "{article}"')
print(f'df_questions.loc[index, "text"] = "{model_questions}"')




##############
Article 99 Entry into force and application
    1. This Regulation shall enter into force on the twentieth day following that of its publication in the Official Journal of the European Union.
    2. It shall apply from 25 May 2018.

##############
df_summary.loc[index, "article"] = "99"
df_summary.loc[index, "text"] = "This regulation became effective 20 days after its publication in the Official Journal of the European Union and has been applicable since 25 May 2018."

df_questions.loc[index, "article"] = "99"
df_questions.loc[index, "text"] = "When did GDPR come into effect?|What is the timeline for GDPR becoming applicable?"


In [93]:
df_summary.loc[index, "article"] = "99"
df_summary.loc[index, "text"] = "This regulation became effective 20 days after its publication in the Official Journal of the European Union and has been applicable since 25 May 2018."

df_questions.loc[index, "article"] = "99"
df_questions.loc[index, "text"] = "When did GDPR come into effect?|What is the timeline for GDPR becoming applicable?"



index = index + 1
if index == len(sectioned_df):
    print("All done!")
else:
    next_section = sectioned_df.iloc[index]["article"]
    assert len(sectioned_df[sectioned_df["article"] == next_section]) == 1, "Huston, we have a problem"
    print(f'Next section is {next_section} which is on line {index}')
    p = ((index) / len(sectioned_df)) * 100
    print(f"You have completed {p:.2f} percent of your work")
    reg_text = sectioned_df.loc[index]['text']
    print("Next section")
    print("##############")
    print(reg_text)
    print("##############")

All done!


In [26]:
index = index + 1


In [94]:
# Sometimes there are errors in the previous code block. We need to be careful when saving over any work we have already done so the 
# save step is a manual one which needs to be run regularly but without overwriting good data with bad data
df_summary.to_parquet(article_summary_with_embeddings, engine='pyarrow')
df_questions.to_parquet(article_questions_with_embeddings, engine='pyarrow')


In [95]:
df_summary


,text,article
0,This Regulation establishes rules for the prot...,1
1,This Regulation applies to the processing of p...,2
2,The Regulation applies to the processing of pe...,3
4,"Personal data must be processed lawfully, fair...",5
5,Processing of personal data is only lawful if ...,6
...,...,...
94,This Regulation won't add extra obligations on...,95
95,International agreements made by Member States...,96
96,The European Commission is required to evaluat...,97
97,The Commission may propose changes to other EU...,98


In [96]:
articles = df_summary['article'].to_list()
articles_int = [int(article) for article in articles]
has_duplicates = len(articles) != len(set(articles))
print(f'has_duplicates : {has_duplicates}')
missing_numbers = set(range(1, 71)) - set(articles_int)
print(f'missing_numbers : {missing_numbers}')


has_duplicates : False
missing_numbers : {4}


## Create the index file

In [122]:
# article_summary_with_embeddings 
# article_questions_with_embeddings

df_index = pd.read_parquet(article_questions_with_embeddings, engine='pyarrow')
df_index = df_index[df_index["text"] != ""] # remove rows that have 'text' == ""
print(len(df_index))
# the 'text' column for the questions may contain multiple questions separated by a "|". The next line expands these rows
# so the value in 'text' only contains one question
df_index = df_index.drop("text", axis=1).join(df_index["text"].str.split("|", expand=True).stack().reset_index(level=1, drop=True).rename("text"))
df_index.reset_index(drop=True, inplace=True)
print(len(df_index))
df_index["source"] = "question"

df_tmp = pd.read_parquet(article_summary_with_embeddings , engine='pyarrow')
df_tmp["source"] = "summary"

df_index = pd.concat([df_index, df_tmp], ignore_index = True)

98
550


In [124]:
print(len(df_index))
df_index = df_index[df_index["text"]!= ""]
df_index = df_index[df_index["text"].notna()] # Remove any NaN's
df_index.reset_index(drop=True, inplace=True)
print(len(df_index))

index_file = './inputs/index_gdpr.parquet'
df_index.to_parquet(index_file, engine = 'pyarrow')



647
647


In [34]:
from regulations_rag.standard_regulation_index import load_parquet_data, save_parquet_data
import os
key = os.getenv('encryption_key_gdpr')
#dfn_index = load_parquet_data("../inputs/definitions_gdpr.parquet")
index_df = load_parquet_data("../inputs/index/gdpr.parquet", key)



In [35]:
dfns = index_df[index_df['source'] == "definitions"]
#index

In [11]:
import re
def extract_quoted_text(s):
    matches = re.findall(r"'([^']*)'", s)
    return matches[0] if matches else ""

# Create list using the extracted text
what_is_list = ["What is " + extract_quoted_text(text) + "?" for text in dfns['text']]

In [36]:
what_is_list = ['What is personal data?',
 'What is processing?',
 'What is restriction of processing?',
 'What is profiling?',
 'What is pseudonymisation?',
 'What is a filing system?',
 'What is a controller?',
 'What is a processor?',
 'What is a recipient?',
 'What is a third party?',
 'What is consent?',
 'What is a personal data breach?',
 'What is genetic data?',
 'What is biometric data?',
 'What is data concerning health?',
 'What is a main establishment?',
 'What is a representative?',
 'What is an enterprise?',
 'What is a group of undertakings?',
 'What are binding corporate rules?',
 'What is supervisory authority?',
 'What is supervisory authority concerned?',
 'What is cross-border processing?',
 'What is relevant and reasoned objection?',
 'What is information society service?',
 'What is international organisation?']

In [13]:
text_to_change = dfns.iloc[0]["text"]
text_to_change

"'personal data' means any information relating to an identified or identifiable natural person ('data subject'); an identifiable natural person is one who can be identified, directly or indirectly, in particular by reference to an identifier such as a name, an identification number, location data, an online identifier or to one or more factors specific to the physical, physiological, genetic, mental, economic, cultural or social identity of that natural person;"

In [38]:
from openai import OpenAI
openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"),)

import sys
sys.path.append('E:/Code/chat/gdpr')

from src.index_tools import update_text_in_index

model = "text-embedding-3-large"
dimensions = 1024
counter = 1
for index, row in dfns.iterrows():
    update_text_in_index(openai_client = openai_client, index_df = index_df, text_to_change = dfns.iloc[counter]["text"], changed_text = what_is_list[counter], embedding_model = model, embedding_dimensions = dimensions)
    counter = counter + 1

IndexError: single positional indexer is out-of-bounds

In [41]:
index_df[20:30]

,text,embedding,source,section_reference,document
20,What is supervisory authority?,"[-0.02389775589108467, -0.02977169118821621, -...",definitions,4(21),GDPR
21,What is supervisory authority concerned?,"[-0.016888560727238655, 0.003025628859177232, ...",definitions,4(22),GDPR
22,What is cross-border processing?,"[-0.03551863506436348, -3.7787705764458224e-07...",definitions,4(23),GDPR
23,What is relevant and reasoned objection?,"[-0.01737261191010475, -0.005801921710371971, ...",definitions,4(24),GDPR
24,What is information society service?,"[-0.04533061757683754, 0.06807615607976913, -0...",definitions,4(25),GDPR
25,What is international organisation?,"[-0.05958525463938713, -0.022743836045265198, ...",definitions,4(26),GDPR
26,What are the main purposes of GDPR?,"[-0.06360988318920135, -0.05781147629022598, -...",question,1,GDPR
27,What rights and freedoms does GDPR protect for...,"[-0.07677329331636429, -0.0514022633433342, -0...",question,1,GDPR
28,Is the movement of personal data allowed withi...,"[-0.06736818701028824, -0.009348609484732151, ...",question,1,GDPR
29,What does GDPR apply to?,"[-0.08824595808982849, 0.0009544518543407321, ...",question,2,GDPR


In [42]:
save_parquet_data(index_df, "../inputs/index/gdpr.parquet", key)

In [12]:
index.loc[index['document'] == 'gdpr', 'document'] = 'GDPR'

In [14]:
save_parquet_data(index, "../inputs/index/gdpr.parquet", key)

In [24]:
dfn_index['section_reference'] = dfn_index.index.to_series().apply(lambda x: f"4({x+1})")

In [25]:
dfn_index.rename(columns={'definition': 'text'}, inplace=True)


In [26]:
dfn_index.drop('term', axis=1, inplace=True)

In [27]:
dfn_index['source'] = 'definitions'

In [29]:
dfn_index.to_parquet("../inputs/definitions_gdpr.parquet", engine = "pyarrow")

In [34]:
import pandas as pd
combined = pd.concat([dfn_index, index], ignore_index=False)

In [36]:
combined["document"] = "gdpr"

In [41]:
gdpr_index_file = "../inputs/gdpr.index"
save_parquet_data(combined, gdpr_index_file, key)
#combined.to_parquet(gdpr_index_file, engine = 'pyarrow')